<a href="https://colab.research.google.com/github/The-DigitalAcademy/TrendHunt-Application/blob/Streamlit/main_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 23.8 MB/s eta 0:00:00


In [13]:
!wget -q -O - ipv4.icanhazip.com


35.196.210.179


In [14]:
%%writefile app.py
import streamlit as st
import os
import cv2
import keras
import numpy as np
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, decode_predictions, preprocess_input
from PIL import Image
import pickle
from scipy.spatial import distance

# Load the pre-trained VGG16 model
model = VGG16(weights='imagenet')

# Load the saved PCA features and images list
file_path = '/content/drive/MyDrive/my data.p'
images, pca_features, pca = pickle.load(open(file_path, 'rb'))

# Define the feature extractor using the VGG16 model
base_model = VGG16(weights='imagenet')
layer_name = 'fc2'
feat_extractor = keras.models.Model(inputs=base_model.input, outputs=base_model.get_layer(layer_name).output)

# Function to load and preprocess an image
def load_image(path):
    img = image.load_img(path, target_size=(224, 224))

    # Ensure the image is in RGB mode
    if img.mode != "RGB":
        img = img.convert("RGB")

    x = image.img_to_array(img)
    x = preprocess_input(x)
    x = np.expand_dims(x, axis=0)
    return x

# Streamlit UI
st.title("Image Classifier with Streamlit")

# Option to upload an image
uploaded_image = st.file_uploader("Upload an image...", type=["jpg", "png", "jpeg"])

# Option to capture an image using the camera
if st.button("Capture Image from Camera"):
    st.write("Camera Opened")
    camera_image = st.image([], caption="Camera Feed", use_column_width=True, channels="BGR")

    # Create a unique filename for the captured image
    img_path = "captured_image.jpg"

    # Capture an image from the camera and save it
    camera = cv2.VideoCapture(0)
    ret, frame = camera.read()
    if ret:
        cv2.imwrite(img_path, cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        camera_image.image(frame, caption="Captured Image", use_column_width=True)
        st.write("Image Captured")
    else:
        st.write("Error capturing image from camera")
    camera.release()

if uploaded_image is not None:
    # Display the uploaded image
    st.image(uploaded_image, caption="Uploaded Image", use_column_width=True)

    # Perform predictions on the uploaded image
    img = Image.open(uploaded_image)

    # Convert the image to RGB mode explicitly
    img = img.convert("RGB")

    img_path = "temp_image.jpg"
    img.save(img_path)

    # Load and preprocess the uploaded image
    x = load_image(img_path)

    # Make predictions using the model
    predictions = model.predict(x)

    # Decode the predictions to get class labels and probabilities
    decoded_predictions = decode_predictions(predictions, top=5)[0]

    st.subheader("Top Predicted Classes:")
    for label, description, score in decoded_predictions:
        st.write(f"{description} ({label}): {score:.2f}")

    # Calculate PCA features for the uploaded image
    img_features = feat_extractor.predict(x)[0]
    img_pca_features = pca.transform([img_features])[0]

    # Calculate distances to all other images
    distances = [distance.cosine(img_pca_features, feat) for feat in pca_features]
    idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])[0:5]

    st.subheader("Most Similar Images:")
    for idx in idx_closest:
        similar_img = Image.open(images[idx])
        st.image(similar_img, caption=f"Similar Image {idx}", use_column_width=True)


Overwriting app.py


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.210.179:8501

npx: installed 22 in 3.338s
your url is: https://tidy-lines-attack.loca.lt
2023-09-19 08:31:09.166449: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-19 08:31:11.233189: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-19 08:31:35.263 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 552, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 